In [2]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [3]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [4]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [5]:
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                'ionosphere.mat',
                'letter.mat',
                'lympho.mat',
                'mnist.mat',
                'musk.mat',
                'optdigits.mat',
                'pendigits.mat',
                'pima.mat',
                'satellite.mat',
                'satimage-2.mat',
                'shuttle.mat',
                'vertebral.mat',
                'vowels.mat',
                'wbc.mat']

In [6]:
data=loadmat('data/cardio.mat')

In [7]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [9]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

In [10]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

In [ ]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))
    
    X = mat['X']
    y = mat['y'].ravel()
    outliners_fraction = np.count_nonzero(y)/len(y)
    outliners_percentage = round(outliners_fraction*100, ndigits=4)
    
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliners_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliners_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliners_percentage]
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=random_state)
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {'Angle-based outlier Dectector (ABOD)': ABOD(contamination=outliners_fraction),
                  'Cluster-based Local Outlier Factor': CBLOF(contamination=outliners_fraction,check_estimator=False,random_state=random_state),
                  'FeatureBagging':FeatureBagging(contamination=outliners_fraction,random_state=random_state),
                  'Histogram-base Outier Dectection': HBOS(contamination=outliners_fraction),
                  'Isolation Forest':IForest(contamination=outliners_fraction,random_state=random_state),
                  'KNN':KNN(contamination=outliners_fraction),
                  'Local Outlier Factor':LOF(contamination=outliners_fraction),
                  'MCD':MCD(contamination=outliners_fraction,random_state=random_state),
                  'OCSVM':OCSVM(contamination=outliners_fraction),
                  'PCA':PCA(contamination=outliners_fraction,random_state=random_state),
                  }
    
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_test_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1=time()
        duration=round(t1-t0, ndigits=4)
        time_list.append(duration)
        
        roc=round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
        
        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, execution time:{duration}'.format(clf_name=clf_name, roc=roc, prn=prn,duration=duration ))
        
        roc_list.append(roc)
        prn_list.append(prn)
        
temp_df = pd.DataFrame(time_list).transpose()
temp_df.columns = df_columns
time_df = pd.concat([time_df, temp_df], axis=0)

temp_df = pd.DataFrame(roc_list).transpose()
temp_df.columns = df_columns
time_df = pd.concat([roc_df, temp_df], axis=0)

temp_df = pd.DataFrame(prn_list).transpose()
temp_df.columns = df_columns
time_df = pd.concat([prn_df, temp_df], axis=0)
   